<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches

在这个实验中，你将进行网络爬取，从一个名为“Falcon 9和Falcon Heavy发射列表”的维基百科页面收集Falcon 9的历史发射记录。

链接为：https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches

详细解释：

网络爬取是一个通过自动或半自动方式从互联网上获取信息的过程。在这个实验中，你将使用Python进行网络爬取，从维基百科的页面上获取Falcon 9火箭的发射历史数据。

Falcon 9是由SpaceX公司开发的一种重型可回收火箭，是目前全球最活跃的商业运载火箭。通过收集其历史发射记录，我们可以分析其发射成功率、载荷能力、回收情况等多方面的信息，以便更好地了解其性能和商业价值。

在这个实验中，你将主要使用到Python的requests库来发送HTTP请求，BeautifulSoup库来解析HTML页面，以及pandas库来处理和存储数据。

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


#### 下面函数 `date_time` 接受一个 HTML 表格单元格的元素作为输入，并从中提取日期和时间信息。

该函数的内部逻辑是这样的：

1. `table_cells.strings`：这一步将从表格单元格中提取所有的文本内容。在 BeautifulSoup 中，`.strings` 属性可以获取到一个元素下的所有文本内容，包括其子孙元素的文本内容，返回一个生成器。

2. `list(table_cells.strings)`：这一步将生成器转化为一个列表，列表中的每一项都是表格单元格中的一段文本。

3. `[data_time.strip() for data_time in list(table_cells.strings)]`：这是一个列表推导式，用于遍历刚刚获取的文本列表，对每一段文本执行 `.strip()` 操作。`.strip()` 是 Python 中的一个字符串方法，用于移除字符串首尾的空白字符，包括空格、制表符、换行符等。

4. `[data_time.strip() for data_time in list(table_cells.strings)][0:2]`：最后，这一步取出经过处理的列表中的前两项，即日期和时间信息。

总的来说，这个函数的作用就是从 HTML 表格单元格中提取出日期和时间信息，并去除其首尾的空白字符。

In [5]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

下面函数 `booster_version` 的目的是从 HTML 表格单元格中提取火箭（booster）的版本信息。

函数的内部逻辑是这样的：

1. `table_cells.strings`：这一步将从表格单元格中提取所有的文本内容。在 BeautifulSoup 中，`.strings` 属性可以获取到一个元素下的所有文本内容，包括其子孙元素的文本内容，返回一个生成器。

2. `enumerate( table_cells.strings)`：`enumerate` 函数用于将一个可遍历的数据对象（如列表、元组或字符串）组合为一个索引序列，同时列出数据和数据下标，一般用在 for 循环当中。

3. `[booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0]`：这是一个列表推导式，用于遍历刚刚获取的带有索引的文本列表，对每一段文本进行判断，如果其索引为偶数（即 `i%2==0`）就将其加入新的列表。

4. `''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])`：最后，这一步将筛选后的文本列表合并为一个字符串。`[0:-1]` 用于取出列表中的第一项到倒数第二项（不包括最后一项），然后使用 `''`（空字符串）作为连接符将这些文本拼接起来。

总的来说，这个函数的作用就是从 HTML 表格单元格中提取出火箭的版本信息，并将其组合成一个字符串。

"新的列表"是指由列表推导式 `[booster_version for i,booster_version in enumerate(table_cells.strings) if i%2==0]` 生成的列表。

这个列表推导式会遍历从 `table_cells.strings` 得到的带有索引的文本列表。如果文本的索引 `i` 是偶数（即 `i%2==0`），那么这个文本 `booster_version` 就会被添加到新的列表中。

这是一个 Python 的列表推导式，它可以用一行代码生成一个列表。在这个例子中，列表推导式的条件是 `i%2==0`，意思是只有当索引 `i` 是偶数时，对应的文本 `booster_version` 才会被添加到新的列表中。

总的来说，这个列表推导式会创建一个新的列表，其中只包含 `table_cells.strings` 中索引为偶数的文本元素。这个新的列表就是我们提到的 "新的列表"。

In [6]:
def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out 

下面函数 `landing_status(table_cells)` 的目的是从 HTML 表格单元格中获取着陆状态。

函数的输入 `table_cells` 是一个 BeautifulSoup 对象，表示一个 HTML 表格单元格的内容。

函数内部的列表推导式 `[i for i in table_cells.strings]` 会遍历 `table_cells.strings` 中的每一个字符串 `i`，并将它们全部收集到一个新的列表中。这个过程就是提取表格单元格中所有的字符串内容。

然后，`[0]` 会获取这个新列表的第一个元素，也就是表格单元格中的第一条字符串，通常这就是我们需要的着陆状态。

所以，这个函数的作用是从 HTML 表格单元格中提取并返回着陆状态。

In [7]:
def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out

下面 `get_mass(table_cells)` 函数的目的是从 HTML 表格单元格中获取负载质量。

函数的输入 `table_cells` 是一个 BeautifulSoup 对象，表示一个 HTML 表格单元格的内容。

首先，`unicodedata.normalize("NFKD", table_cells.text).strip()` 这行代码的作用是对表格单元格中的文本进行规范化处理。规范化是将 Unicode 字符串转换为几乎相等的等效字符串，但具有更规范的形式的过程。`"NFKD"` 是规范化的一种形式，它将字符转换为等效的兼容分解。然后，`.strip()` 方法会移除字符串头尾的空白字符。

然后，`mass.find("kg")` 会找到 "kg" 在字符串 `mass` 中的位置。

接着，`new_mass=mass[0:mass.find("kg")+2]` 会截取从开始到 "kg" 后两个字符的字符串，这通常就是我们需要的负载质量。

如果 `mass` 为空，那么 `new_mass` 就会被设为 0。

所以，这个函数的作用是从 HTML 表格单元格中提取并返回负载的质量。

在Python中，字符串是一个序列，我们可以通过索引访问和提取其各个部分。语法 `str[start: end]` 用于截取字符串的一部分，其中 `start` 是开始索引，`end` 是结束索引。

`mass.find("kg")` 会返回 "kg" 在字符串 `mass` 中首次出现的位置索引。假设我们有一个字符串 `mass = '4300 kg'`，那么 `mass.find("kg")` 会返回 5，这是 "kg" 开始的位置。因此，`mass[0: mass.find("kg")]` 将会截取从索引0开始到索引5的字符串，即 '4300 '。

然而，我们希望保留 "kg"，因此我们需要在 `mass.find("kg")` 的结果上加 2（因为 "kg" 有两个字符）。所以，`mass[0: mass.find("kg") + 2]` 将会截取从索引0开始到索引7的字符串，即 '4300 kg'。

在这个例子中，`new_mass = mass[0: mass.find("kg") + 2]` 的结果将会是 '4300 kg'，这就是我们需要的负载质量。

In [2]:
def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

下面函数 `extract_column_from_header` 用于从HTML表格的表头行中提取列名。

首先，我们会去除包含在行元素中的 `<br>`，`<a>` 和 `<sup>` 标签，这些标签通常会带来我们不需要的额外信息。`row.br.extract()`、`row.a.extract()` 和 `row.sup.extract()` 这些操作会移除对应的HTML标签及其内容。

然后，`' '.join(row.contents)` 将会把行元素中的所有内容连接成一个字符串，各部分之间以空格隔开。

接下来的 `if` 语句是用来过滤掉只包含数字或者为空的列名。如果 `colunm_name.strip().isdigit()` 为 `True`，说明列名只包含数字，这样的列名我们不需要。如果 `colunm_name.strip()` 为空字符串，也说明这不是一个有效的列名。

最后，如果列名既不全是数字也不为空，我们就返回经过 `strip()` 函数处理过的列名。`strip()` 函数会移除字符串前后的空白字符，包括空格、制表符、换行符等。这样我们就得到了一个干净整洁的列名。

In [3]:
def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name  

To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`

为了保持实验任务的一致性，你将被要求从2021年6月9日更新的 "Falcon 9和Falcon Heavy发射列表" 的Wikipedia页面的快照中爬取数据。

这句话的意思是，你需要爬取的网页数据并非是实时的最新数据，而是2021年6月9日时的网页数据的一个快照。这是为了确保所有进行此实验的人爬取到的数据是一致的，避免因网页内容的实时变动导致的数据不一致问题。

网页数据的快照，也被称为网页存档或网页备份，是某一特定时间点网页内容的完整复制版本。这就像是在某个时间点对网页进行拍照，记录下了那一刻网页的所有信息。

网页快照通常用于以下情况：

1. **网页内容的历史记录**：网页的内容可能会随着时间的推移而变化，有时原来的信息可能会被删除或修改。通过创建网页的快照，可以保存网页在特定时间点的状态，以便日后查阅。

2. **数据一致性**：在进行数据分析或机器学习项目时，可能需要确保所有的参与者或学习者获取到的数据是一致的。在这种情况下，可以使用某一时间点的网页快照，使得大家访问到的都是同一份数据，防止由于网页内容的实时更新导致数据不一致。

3. **网页内容的备份**：如果某个网页在未来可能无法访问，但你又希望保存其内容，那么可以创建该网页的快照并保存下来。

在你的情况下，使用网页的快照可以确保你在进行实验时使用的数据是一致的，不会因为Wikipedia页面的实时更新而受到影响。

In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
# use requests.get() method with the provided static_url
# assign the response to a object
response = requests.get(static_url)
response.status_code

200

Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
from bs4 import BeautifulSoup

`BeautifulSoup` 是一个用于从HTML和XML文件中提取数据的Python库。它为我们提供了一种方便的方式来解析HTML和XML文档，并从中提取所需的信息。

下面是对`soup = BeautifulSoup(response.content, 'html.parser')`这一行代码的详细解释：

1. **`response.content`**:
    - 这通常来自`requests`库的响应对象，其中`.content`返回服务器响应的内容，即网页的HTML源码。这里假设你已经使用了`requests`库发送了一个HTTP请求并得到了一个响应。
  
2. **`'html.parser'`**:
    - 这是你想要使用的解析器的名称。`BeautifulSoup`支持多种解析器，其中`html.parser`是Python的标准库中内置的解析器。它足够适合大多数任务，但在某些情况下，你可能想要使用其他的解析器，例如`lxml`或`html5lib`。
  
3. **`BeautifulSoup`**:
    - 这是`bs4`库中的主要类，它允许你创建一个BeautifulSoup对象。当你创建这个对象时，它将开始解析你提供的HTML或XML内容。
  
4. **`soup`**:
    - 这是你创建的BeautifulSoup对象的变量名。一旦你有了这个对象，你可以使用它来查询、修改和/或提取HTML文档中的特定部分。

以下是一个简单示例，展示如何使用`BeautifulSoup`和`requests`库来从一个网页中提取标题:

```python
import requests
from bs4 import BeautifulSoup

url = "https://www.example.com"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

title = soup.title.string
print(title)
```

在上面的示例中，我们首先发送一个HTTP GET请求到`https://www.example

In [7]:
# 使用 BeautifulSoup 解析 HTML 响应
soup = BeautifulSoup(response.content, 'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [8]:
# Use soup.title attribute
print(soup.title.string)

List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table headerNext, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


接下来，我们想从HTML表格标题中收集所有相关的列名。首先，让我们尝试找到wiki页面上的所有表格。如果你需要回顾一下BeautifulSoup的知识，请查看本实验室末尾的外部参考链接。

在这个步骤中，我们会使用BeautifulSoup库来解析HTML页面，找出页面上所有的表格。每个HTML表格都是由一个`<table>`标签开始的，然后包含多个`<tr>`（表格行）标签，每个`<tr>`标签又包含多个`<th>`或`<td>`标签，分别表示表头和表格数据。所以，我们可以通过在BeautifulSoup对象上调用`find_all('table')`方法，来找出页面上所有的表格。然后我们可以进一步处理每个表格，提取出我们需要的数据。

In [10]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')

Starting from the third table is our target table contains the actual launch records.


在下面代码中，`first_launch_table` 被赋值为 `html_tables` 列表的第三个元素。在 Python 中，列表的索引是从 0 开始的，所以 `html_tables[2]` 实际上是指向列表中的第三个元素。
这意味着我们正在获取网页上的第三个 HTML 表格，并将其存储在 `first_launch_table` 变量中。这个表格可能包含一些特定的数据，例如火箭的首次发射数据。具体内容会根据你正在处理的网页的实际内容而变化。

In [11]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


接下来，我们只需要遍历`<th>`元素，并应用提供的`extract_column_from_header()`函数来逐个提取列名。

详细解释：
1. **遍历`<th>`元素**:
   HTML表格通常由`<table>`标签定义。在表格中，`<th>`标签用于定义头部单元格，它通常表示列的标题或名称。遍历`<th>`元素意味着我们要按顺序查看每一个这样的头部单元格。

2. **应用`extract_column_from_header()`函数**:
   这意味着对每一个遍历到的`<th>`元素，我们都会调用`extract_column_from_header()`这个函数，并将该`<th>`元素作为参数传递（或其内容，具体取决于函数的设计）。这个函数的目的可能是从`<th>`元素中提取特定的文本或数据，例如列的名称或其他相关信息。

3. **逐个提取列名**:
   当我们遍历表格的每一个`<th>`元素并使用`extract_column_from_header()`函数时，我们会逐个得到每一列的名称。这可能是为了创建一个列名的列表，或者为了其他一些目的，如数据整理或分析。

总之，这句话的意思是：对于HTML表格中的每一个列标题（由`<th>`标签定义），我们都会使用一个特定的函数`extract_column_from_header()`来提取该列的名称或标题。

In [17]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
# Apply find_all() function with `th` element on first_launch_table
table_headers = first_launch_table.find_all('th')

for th in table_headers:
    # Apply the provided extract_column_from_header() to get a column name
    column_name = extract_column_from_header(th)
    
    # Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
    if column_name is not None and len(column_name) > 0:
        column_names.append(column_name)


Check the extracted column names


In [18]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


这段代码首先创建一个字典，并然后修改和扩展这个字典。下面是对代码的详细解释：

1. **创建一个新的字典**:
```python
launch_dict= dict.fromkeys(column_names)
```
这行代码使用 `dict.fromkeys()` 方法创建一个新的字典。这个方法将 `column_names` 列表中的每个项作为字典的键，并为每个键分配一个默认的 `None` 值。因此，如果 `column_names` 是 `['a', 'b', 'c']`，那么 `launch_dict` 将为 `{ 'a': None, 'b': None, 'c': None }`。

2. **删除一个不相关的列**:
```python
del launch_dict['Date and time ( )']
```
这行代码使用 `del` 语句从 `launch_dict` 字典中删除 `'Date and time ( )'` 这个键及其对应的值。

3. **初始化 `launch_dict`**:
接下来的几行代码将 `launch_dict` 字典中的特定键的值设置为一个空列表。例如：
```python
launch_dict['Flight No.'] = []
```
这行代码将 `'Flight No.'` 这个键的值设置为一个空列表。

4. **添加新的列**:
最后几行代码为 `launch_dict` 添加了一些新的键，并为它们分配了空列表作为值。

综上所述，这段代码的目的是初始化一个字典，该字典表示某种启动数据（可能是火箭发射）。每个键代表一个数据列，而值（在这里是空列表）将用于存储该列的数据。

In [20]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


翻译：
接下来，我们只需要用从表格行中提取的发射记录填充`launch_dict`。

通常，Wiki页面中的HTML表格可能包含意外的注解和其他类型的噪音，如参考链接B0004.1[8]、缺失值N/A [e]、不一致的格式等。

为了简化解析过程，我们在下面提供了一个不完整的代码片段，以帮助你填充`launch_dict`。请完成以下带有TODOs的代码片段，或者你可以选择编写自己的逻辑来解析所有的发射表格：

详细解释：
1. **填充`launch_dict`**:
    该文段首先提到了一个即将进行的任务，即使用从HTML表格行中提取的数据来填充`launch_dict`字典。

2. **HTML表格中的噪音**:
    在很多Wiki页面的HTML表格中，可能会出现一些我们不希望解析的数据，或者说噪音。这些噪音可能是参考链接（例如`B0004.1[8]`）、缺失值（例如`N/A [e]`）或不一致的格式。

3. **简化解析过程**:
    由于HTML表格中可能存在各种不一致性和噪音，解析这些表格可能会很复杂。为了帮助读者简化这个过程，提供了一个不完整的代码片段。这个片段可能已经包含了一些基本的解析逻辑，但还需要用户补充和完成。

4. **完成代码或编写自己的逻辑**:
    最后，读者被给予选择：他们可以按照给定的指示（TODOs）完成提供的代码片段，或者完全从头开始，使用自己的逻辑来解析表格。

这段文字基本上是一个介绍和指导，它描述了解析Wiki页面中的HTML表格所面临的挑战，并提供了一个部分完成的代码片段来帮助解决这些问题。

#### 下面代码片段旨在从一个Wiki页面中提取火箭发射相关的数据。该页面使用了带有特定CSS类的HTML表格来存储这些信息。下面是对代码的详细解释：

1. **初始化**:
```python
extracted_row = 0
```
这行代码初始化了一个变量 `extracted_row`，用于计数提取的行数。

2. **遍历每个表格**:
```python
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
```
使用`enumerate`和`soup.find_all`遍历页面上的所有匹配指定CSS类的表格。

3. **提取表格行**:
```python
for rows in table.find_all("tr"):
```
这里，我们遍历当前表格中的每一行。

4. **检查航班号**:
以下代码检查行的第一个表头是否是一个数字（代表航班号）：
```python
if rows.th:
    if rows.th.string:
        flight_number = rows.th.string.strip()
        flag = flight_number.isdigit()
else:
    flag = False
```
如果该行的第一个表头是数字，`flag` 设置为`True`，否则为`False`。

5. **提取行数据**:
如果 `flag` 为 `True`（表示当前行确实包含发射数据），则执行以下操作：
- 递增 `extracted_row`，表示已提取的行数。
- 提取与火箭发射相关的各种数据，如航班号、日期、时间、助推器版本、发射地点等。每次提取都使用特定的方法或逻辑。
  
  例如：
```python
datatimelist = date_time(row[0])
```
这里使用了 `date_time` 函数（在代码片段中没有提供，但可能在其他地方定义）来处理和提取日期和时间信息。

6. **TODOs**:
在代码中，有许多 `TODO` 注释，提示你需要将提取的数据追加到之前初始化的 `launch_dict` 字典中。例如：
```python
# TODO: Append the flight_number into launch_dict with key `Flight No.`
```
这个 `TODO` 提示你应该将提取的 `flight_number` 添加到 `launch_dict` 字典的 `Flight No.` 键下。

总的来说，这个代码片段是为了从Wiki页面上的特定表格中提取火箭发射的相关数据，并将这些数据存储在一个字典中。它使用了BeautifulSoup库来解析HTML，并提取所需的数据。

#### 该错误是由于在尝试访问一个`None`对象的`.string`属性时产生的。具体来说，这是因为你试图访问`row[6].a`，但`row[6]`可能没有`<a>`标签，导致返回`None`。然后你尝试从这个`None`对象中获取`.string`属性，引发了错误。

为了修复这个问题，我们需要增加额外的检查以确保`<a>`标签存在。

修改如下：

```python
if row[6].a:  # Check if the <a> tag exists
    customer = row[6].a.string
else:
    customer = row[6].string  # Fallback to the text within the <td> tag
```

整合到你的代码中：

```python
...
# Customer
# TODO: Append the customer into launch_dict with key `Customer`
if row[6].a:  # Check if the <a> tag exists
    customer = row[6].a.string
else:
    customer = row[6].string  # Fallback to the text within the <td> tag
#print(customer)
...
```

这样，即使`row[6]`中没有`<a>`标签，代码也会尝试从`<td>`标签中提取文本，从而避免了错误。确保其他可能出现相似问题的地方也进行类似的检查。

In [24]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            # print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            if row[6].a:  # Check if the <a> tag exists
                customer = row[6].a.string
            else:
                customer = row[6].string  # Fallback to the text within the <td> tag

            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)            

In [26]:
extracted_row = 0
launch_dict = {}  # initialize an empty dictionary

#Extract each table 
for table_number, table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
    # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            launch_dict[extracted_row] = {}  # create a new dictionary for each row
            
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            launch_dict[extracted_row]['Flight No.'] = flight_number
            
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            launch_dict[extracted_row]['Date'] = datatimelist[0].strip(',')
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            launch_dict[extracted_row]['Time'] = datatimelist[1]
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict[extracted_row]['Version Booster'] = bv
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_dict[extracted_row]['Launch Site'] = row[2].a.string
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            launch_dict[extracted_row]['Payload'] = row[3].a.string
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            launch_dict[extracted_row]['Payload mass'] = get_mass(row[4])
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            launch_dict[extracted_row]['Orbit'] = row[5].a.string
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            if row[6].a:  # Check if the <a> tag exists
                launch_dict[extracted_row]['Customer'] = row[6].a.string
            else:
                launch_dict[extracted_row]['Customer'] = row[6].string  # Fallback to the text within the <td> tag
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_dict[extracted_row]['Launch outcome'] = list(row[7].strings)[0]
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            launch_dict[extracted_row]['Booster landing'] = landing_status(row[8])


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [29]:
df=pd.DataFrame(launch_dict)
df.head()

,1,2,3,4,5,6,7,8,9,10,...,112,113,114,115,116,117,118,119,120,121
Flight No.,1,2,3,4,5,6,7,8,9,10,...,112,113,114,115,116,117,118,119,120,121
Date,4 June 2010,8 December 2010,22 May 2012,8 October 2012,1 March 2013,29 September 2013,3 December 2013,6 January 2014,18 April 2014,14 July 2014,...,24 March 2021,7 April 2021,23 April 2021,29 April 2021,4 May 2021,9 May 2021,15 May 2021,26 May 2021,3 June 2021,6 June 2021
Time,18:45,15:43,07:44,00:35,15:10,16:00,22:41,22:06,19:25,15:15,...,08:28,16:34,9:49,03:44,19:01,06:42,22:56,18:59,17:29,04:26
Version Booster,F9 v1.0B0003.1,F9 v1.0B0004.1,F9 v1.0B0005.1,F9 v1.0B0006.1,F9 v1.0B0007.1,F9 v1.1B1003,F9 v1.1,F9 v1.1,F9 v1.1,F9 v1.1,...,F9 B5B1060.6,F9 B5 ♺,F9 B5B1061.2,F9 B5B1060.7,F9 B5B1049.9,F9 B5B1051.10,F9 B5B1058.8,F9 B5B1063.2,F9 B5B1067.1,F9 B5
Launch Site,CCAFS,CCAFS,CCAFS,CCAFS,CCAFS,VAFB,CCAFS,CCAFS,Cape Canaveral,Cape Canaveral,...,CCSFS,CCSFS,KSC,CCSFS,KSC,CCSFS,KSC,CCSFS,KSC,CCSFS


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


翻译：
我们现在可以将其导出为CSV，用于下一部分，但为了使答案保持一致，以及以防您在完成此实验时遇到困难。

接下来的实验将使用提供的数据集，使每个实验都独立。

详细解释：
这段话描述了一种处理数据的方法和其背后的原因。

1. **我们现在可以将其导出为CSV，用于下一部分**：这意味着当前的数据或结果可以被保存为一个CSV文件，以备在接下来的步骤或部分中使用。

2. **但为了使答案保持一致，以及以防您在完成此实验时遇到困难**：这句话解释了为什么要这样做。首先，他们希望所有的答案或结果都是一致的，这可能是为了教学的目的，确保每个人都从相同的起点开始。其次，如果某些参与者或学生在实验中遇到问题或困难，他们仍然可以继续进行下一步，而不会因为前面的错误或问题而受到影响。

3. **接下来的实验将使用提供的数据集，使每个实验都独立**：这句话告诉我们，尽管每个实验都是连续的，但它们是独立设计的。这意味着每个实验都有自己的提供的数据集，不依赖于前一个实验的结果。这样设计是为了使每个实验或任务都能独立于其他实验运行，不会因为一个实验中的问题而影响到其他实验。

<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
